Здесь проводится предварительная обработка и очистка сырых данных.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('complete_data.csv', index_col='Unnamed: 0')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5325 entries, 0 to 5324
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             5325 non-null   object 
 1   Episod_duration  5325 non-null   object 
 2   Total_series     5325 non-null   object 
 3   Year             5325 non-null   int64  
 4   Country          5325 non-null   object 
 5   Genres           5325 non-null   object 
 6   Translation      5325 non-null   object 
 7   Score            5325 non-null   float64
 8   Description      5273 non-null   object 
 9   Reviews          5305 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 457.6+ KB


In [4]:
data.head()

,Name,Episod_duration,Total_series,Year,Country,Genres,Translation,Score,Description,Reviews
0,Странная легенда династии Тан дорама (2022),45 мин,36 серий,2022,Китай,"Детектив, Исторический, Фэнтези","Русская озвучка, Русские субтитры",9.04,Странные дела происходят в древней столице Ча...,"мне понравилось! Понравилось, но у меня про..."
1,Письмо с небес дорама (2022),44 мин,26 серий,2022,Китай,"Детектив, Исторический, Комедия, Мелодрама",Русские субтитры,9.39,"Молодая, но уже довольно опытная в жизни Шен ...",г героиня просто бесит Озвучьте кто нибудь....
2,Тринадцать лет забвения дорама (2023),44 мин,24 серии,2023,Китай,"Детектив, Криминал, Триллер",Русские субтитры,9.31,Тринадцать лет назад в городе Наньду произошл...,Отличный сериал!!! Игра актеров была на высше...
3,Шах и мат дорама (2022),45 мин,24 серии,2022,Китай,"Детектив, Исторический, Триллер","Русская озвучка, Русские субтитры",8.96,Молодой любитель юриспруденции и расследовани...,"Очень хорошо дорама просто пушка, Очень краси..."
4,Домашняя школа: ученики под арестом дорама (2023),50 мин,18 серий,2023,Таиланд,"Детектив, Триллер","Русская озвучка, Русские субтитры",9.16,"В специальный интернат, заявленный, как домаш...",Полная чушь…! Аааааа ヽ(♡‿♡)ノ Я так долго жд...


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5325 entries, 0 to 5324
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             5325 non-null   object 
 1   Episod_duration  5325 non-null   object 
 2   Total_series     5325 non-null   object 
 3   Year             5325 non-null   int64  
 4   Country          5325 non-null   object 
 5   Genres           5325 non-null   object 
 6   Translation      5325 non-null   object 
 7   Score            5325 non-null   float64
 8   Description      5273 non-null   object 
 9   Reviews          5305 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 457.6+ KB


# Закодируем жанры

In [6]:
def all_values(data, column):
    genres_list = set()
    for string in range(data.shape[0]):
        genres = data.loc[string, column].split(',')
        for item in genres:
            genres_list.add(item.strip())        
    return genres_list

def code_dict(values_list):
    dictionary = dict()
    for index, item in enumerate(values_list):
        dictionary[item] = index
    return dictionary

def revert_dict(original_dictionary, prefix):
    dictionary_opposite = dict()
    for key, value in original_dictionary.items():
        dictionary_opposite[value] = prefix + str(value)
    return dictionary_opposite

In [7]:
def convert_value(data, dictionary, column):
    new_data = []
    new_item = [0]*len(dictionary)
    for string in range(data.shape[0]):        
        genres = data.loc[string, column].split(',')
        for index, item in enumerate(genres): 
            number = dictionary.get(item.strip(), None)
            if number is not None:
                new_item[number] = 1                
        new_data.append(new_item)
        new_item = [0]*len(dictionary)
    new_data = pd.DataFrame(new_data, columns=dictionary.values())
    return new_data

In [8]:
def transform_column(data, column, column_prefix):
    values_list = list(all_values(data, column))
    dictionary = code_dict(values_list)
    dictionary_opposite = revert_dict(dictionary, column_prefix)
    new_data = convert_value(data, dictionary, column)
    data = pd.concat([data, new_data], axis=1)
    data = data.rename(columns=dictionary_opposite)
    data = data.drop([column], axis=1)
    return dictionary, data

In [9]:
genre_dict, data = transform_column(data, 'Genres', 'genre_')
genre_dict

{'Фантастика': 0,
 'Исторический': 1,
 'Комедия': 2,
 'Детектив': 3,
 'Триллер': 4,
 'Мистика': 5,
 'Военный': 6,
 'Драма': 7,
 'Ток-шоу': 8,
 'Криминал': 9,
 'Боевик': 10,
 'Музыка': 11,
 'Фэнтези': 12,
 'Спорт': 13,
 'Мюзикл': 14,
 'Приключения': 15,
 'Мелодрама': 16,
 'Биография': 17,
 'Боевые искусства': 18,
 'Ужасы': 19,
 'Вестерн': 20}

In [10]:
data.head()

,Name,Episod_duration,Total_series,Year,Country,Translation,Score,Description,Reviews,genre_0,...,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,genre_18,genre_19,genre_20
0,Странная легенда династии Тан дорама (2022),45 мин,36 серий,2022,Китай,"Русская озвучка, Русские субтитры",9.04,Странные дела происходят в древней столице Ча...,"мне понравилось! Понравилось, но у меня про...",0,...,0,1,0,0,0,0,0,0,0,0
1,Письмо с небес дорама (2022),44 мин,26 серий,2022,Китай,Русские субтитры,9.39,"Молодая, но уже довольно опытная в жизни Шен ...",г героиня просто бесит Озвучьте кто нибудь....,0,...,0,0,0,0,0,1,0,0,0,0
2,Тринадцать лет забвения дорама (2023),44 мин,24 серии,2023,Китай,Русские субтитры,9.31,Тринадцать лет назад в городе Наньду произошл...,Отличный сериал!!! Игра актеров была на высше...,0,...,0,0,0,0,0,0,0,0,0,0
3,Шах и мат дорама (2022),45 мин,24 серии,2022,Китай,"Русская озвучка, Русские субтитры",8.96,Молодой любитель юриспруденции и расследовани...,"Очень хорошо дорама просто пушка, Очень краси...",0,...,0,0,0,0,0,0,0,0,0,0
4,Домашняя школа: ученики под арестом дорама (2023),50 мин,18 серий,2023,Таиланд,"Русская озвучка, Русские субтитры",9.16,"В специальный интернат, заявленный, как домаш...",Полная чушь…! Аааааа ヽ(♡‿♡)ノ Я так долго жд...,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
country_dict, data = transform_column(data, 'Country', 'country_')
country_dict

{'Таиланд': 0,
 'Вьетнам': 1,
 'США': 2,
 'Южная Корея': 3,
 'Китай': 4,
 '-': 5,
 'Тайвань': 6,
 'Гонконг': 7,
 'Япония': 8,
 'Филиппины': 9}

In [12]:
data.head()

,Name,Episod_duration,Total_series,Year,Translation,Score,Description,Reviews,genre_0,genre_1,...,country_0,country_1,country_2,country_3,country_4,country_5,country_6,country_7,country_8,country_9
0,Странная легенда династии Тан дорама (2022),45 мин,36 серий,2022,"Русская озвучка, Русские субтитры",9.04,Странные дела происходят в древней столице Ча...,"мне понравилось! Понравилось, но у меня про...",0,1,...,0,0,0,0,1,0,0,0,0,0
1,Письмо с небес дорама (2022),44 мин,26 серий,2022,Русские субтитры,9.39,"Молодая, но уже довольно опытная в жизни Шен ...",г героиня просто бесит Озвучьте кто нибудь....,0,1,...,0,0,0,0,1,0,0,0,0,0
2,Тринадцать лет забвения дорама (2023),44 мин,24 серии,2023,Русские субтитры,9.31,Тринадцать лет назад в городе Наньду произошл...,Отличный сериал!!! Игра актеров была на высше...,0,0,...,0,0,0,0,1,0,0,0,0,0
3,Шах и мат дорама (2022),45 мин,24 серии,2022,"Русская озвучка, Русские субтитры",8.96,Молодой любитель юриспруденции и расследовани...,"Очень хорошо дорама просто пушка, Очень краси...",0,1,...,0,0,0,0,1,0,0,0,0,0
4,Домашняя школа: ученики под арестом дорама (2023),50 мин,18 серий,2023,"Русская озвучка, Русские субтитры",9.16,"В специальный интернат, заявленный, как домаш...",Полная чушь…! Аааааа ヽ(♡‿♡)ノ Я так долго жд...,0,0,...,1,0,0,0,0,0,0,0,0,0


# Получение количества серий и их продолжительности

In [13]:
def convert_digits(data, columns):
    new_data = []
    new_item = []
    for string in range(data.shape[0]):
        for item in columns:            
            new_item.append(get_digits(data.loc[[string], item].array[0])) 
        new_data.append(new_item)
        new_item = []
    new_data = pd.DataFrame(new_data, columns=columns)
    return new_data
            
def get_digits(string):
    word = str(string).strip().split()
    time = 0
    for index in range(len(word)):
        if 'сери' in word[index]:
            if word[index - 1].isdigit():
                return int(float(word[index - 1]))
            elif '(' in word[index - 1]:
                question = word[index - 1].strip('()')
                if question.isdigit():
                    time += int(float(question))
        if 'час' in word[index]:
            if word[index - 1].isdigit():
                time += int(float(word[index - 1])) * 60
        if 'мин' in word[index] or 'миин' in word[index]:
            if word[index - 1].isdigit():
                time += int(float(word[index - 1]))
            elif '-' in word[index - 1]:
                question = word[index - 1].split('-')[0]
                if question.isdigit():
                    time += int(float(question))
    if word[0].isdigit():
        return int(float(word[0]))            
    return time

In [14]:
t = convert_digits(data, ['Episod_duration', 'Total_series'])
t = t.rename(columns={'Episod_duration': 'Episod_duration_2', 'Total_series': 'Total_series_2'})

In [15]:
data = data.drop(['Episod_duration', 'Total_series'], axis=1)

In [16]:
data = pd.concat([data, t], axis=1)

In [17]:
data.tail()

,Name,Year,Translation,Score,Description,Reviews,genre_0,genre_1,genre_2,genre_3,...,country_2,country_3,country_4,country_5,country_6,country_7,country_8,country_9,Episod_duration_2,Total_series_2
5320,Клуб друзей-водителей дорама (2020),2020,Русские субтитры,9.08,В новом шоу «Клуб друзей-водителей» мы познак...,"В этом шоу 26 серий. Шоу очень приятное, ребя...",0,0,0,0,...,0,0,0,0,0,0,0,0,24,9
5321,Почтовый ящик БрайтВинов дорама (2020),2020,Русские субтитры,9.29,Популярные парни из Таиланда — Брайт и Вин бу...,Я их обожаю) все дорамы пересмотрела) но у ме...,0,0,0,0,...,0,0,0,0,0,0,0,0,20,1
5322,Прекрасный вид с Сон Джи Хё дорама (2017),2017,Русские субтитры,8.48,Сон Джи Хё — одна из лучших актрис Южной Коре...,я не знаю но мне нравится shin won ho вы не...,0,0,0,0,...,0,1,0,0,0,0,0,0,54,10
5323,Попался! 2 | Преступник — это ты! 2 дорама (2019),2019,Русские субтитры,8.34,Во втором сезоне детективного ток-шоу «Попалс...,Очень понравился. 100/1000 Омг!!! Шоу потря...,0,0,0,0,...,0,1,0,0,0,0,0,0,1,10
5324,Ночью пятничной пятницы дорама (2020),2020,Русские субтитры,7.56,Каждая серия развлекательно-познавательного т...,Интересно. Очень интересно. Но вот у меня воз...,0,0,0,0,...,0,1,0,0,0,0,0,0,1,11


In [18]:
data = data.rename(columns={'Episod_duration_2': 'Episod_duration', 'Total_series_2': 'Total_series'})

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5325 entries, 0 to 5324
Data columns (total 39 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             5325 non-null   object 
 1   Year             5325 non-null   int64  
 2   Translation      5325 non-null   object 
 3   Score            5325 non-null   float64
 4   Description      5273 non-null   object 
 5   Reviews          5305 non-null   object 
 6   genre_0          5325 non-null   int64  
 7   genre_1          5325 non-null   int64  
 8   genre_2          5325 non-null   int64  
 9   genre_3          5325 non-null   int64  
 10  genre_4          5325 non-null   int64  
 11  genre_5          5325 non-null   int64  
 12  genre_6          5325 non-null   int64  
 13  genre_7          5325 non-null   int64  
 14  genre_8          5325 non-null   int64  
 15  genre_9          5325 non-null   int64  
 16  genre_10         5325 non-null   int64  
 17  genre_11      

In [20]:
data.tail()

,Name,Year,Translation,Score,Description,Reviews,genre_0,genre_1,genre_2,genre_3,...,country_2,country_3,country_4,country_5,country_6,country_7,country_8,country_9,Episod_duration,Total_series
5320,Клуб друзей-водителей дорама (2020),2020,Русские субтитры,9.08,В новом шоу «Клуб друзей-водителей» мы познак...,"В этом шоу 26 серий. Шоу очень приятное, ребя...",0,0,0,0,...,0,0,0,0,0,0,0,0,24,9
5321,Почтовый ящик БрайтВинов дорама (2020),2020,Русские субтитры,9.29,Популярные парни из Таиланда — Брайт и Вин бу...,Я их обожаю) все дорамы пересмотрела) но у ме...,0,0,0,0,...,0,0,0,0,0,0,0,0,20,1
5322,Прекрасный вид с Сон Джи Хё дорама (2017),2017,Русские субтитры,8.48,Сон Джи Хё — одна из лучших актрис Южной Коре...,я не знаю но мне нравится shin won ho вы не...,0,0,0,0,...,0,1,0,0,0,0,0,0,54,10
5323,Попался! 2 | Преступник — это ты! 2 дорама (2019),2019,Русские субтитры,8.34,Во втором сезоне детективного ток-шоу «Попалс...,Очень понравился. 100/1000 Омг!!! Шоу потря...,0,0,0,0,...,0,1,0,0,0,0,0,0,1,10
5324,Ночью пятничной пятницы дорама (2020),2020,Русские субтитры,7.56,Каждая серия развлекательно-познавательного т...,Интересно. Очень интересно. Но вот у меня воз...,0,0,0,0,...,0,1,0,0,0,0,0,0,1,11


# Кодирование категориальных признаков

In [21]:
data['Translation'].unique()

array(['Русская озвучка, Русские субтитры', 'Русские субтитры',
       'Русская озвучка', '-'], dtype=object)

In [22]:
from sklearn.preprocessing import OrdinalEncoder
col = ['Translation']
enc = OrdinalEncoder(categories=[['-', 'Русская озвучка', 'Русская озвучка, Русские субтитры', 'Русские субтитры']])
enc.set_params()
data[col]= enc.fit_transform(data[col])

data.head()

,Name,Year,Translation,Score,Description,Reviews,genre_0,genre_1,genre_2,genre_3,...,country_2,country_3,country_4,country_5,country_6,country_7,country_8,country_9,Episod_duration,Total_series
0,Странная легенда династии Тан дорама (2022),2022,2.0,9.04,Странные дела происходят в древней столице Ча...,"мне понравилось! Понравилось, но у меня про...",0,1,0,1,...,0,0,1,0,0,0,0,0,45,36
1,Письмо с небес дорама (2022),2022,3.0,9.39,"Молодая, но уже довольно опытная в жизни Шен ...",г героиня просто бесит Озвучьте кто нибудь....,0,1,1,1,...,0,0,1,0,0,0,0,0,44,26
2,Тринадцать лет забвения дорама (2023),2023,3.0,9.31,Тринадцать лет назад в городе Наньду произошл...,Отличный сериал!!! Игра актеров была на высше...,0,0,0,1,...,0,0,1,0,0,0,0,0,44,24
3,Шах и мат дорама (2022),2022,2.0,8.96,Молодой любитель юриспруденции и расследовани...,"Очень хорошо дорама просто пушка, Очень краси...",0,1,0,1,...,0,0,1,0,0,0,0,0,45,24
4,Домашняя школа: ученики под арестом дорама (2023),2023,2.0,9.16,"В специальный интернат, заявленный, как домаш...",Полная чушь…! Аааааа ヽ(♡‿♡)ノ Я так долго жд...,0,0,0,1,...,0,0,0,0,0,0,0,0,50,18


# Удаление пропусков

In [24]:
data = data.dropna()

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5255 entries, 0 to 5324
Data columns (total 39 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             5255 non-null   object 
 1   Year             5255 non-null   int64  
 2   Translation      5255 non-null   float64
 3   Score            5255 non-null   float64
 4   Description      5255 non-null   object 
 5   Reviews          5255 non-null   object 
 6   genre_0          5255 non-null   int64  
 7   genre_1          5255 non-null   int64  
 8   genre_2          5255 non-null   int64  
 9   genre_3          5255 non-null   int64  
 10  genre_4          5255 non-null   int64  
 11  genre_5          5255 non-null   int64  
 12  genre_6          5255 non-null   int64  
 13  genre_7          5255 non-null   int64  
 14  genre_8          5255 non-null   int64  
 15  genre_9          5255 non-null   int64  
 16  genre_10         5255 non-null   int64  
 17  genre_11      

In [26]:
data.describe()

,Year,Translation,Score,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,...,country_2,country_3,country_4,country_5,country_6,country_7,country_8,country_9,Episod_duration,Total_series
count,5255.000000,5255.000000,5255.000000,5255.000000,5255.000000,5255.000000,5255.000000,5255.000000,5255.000000,5255.000000,...,5255.000000,5255.000000,5255.000000,5255.000000,5255.000000,5255.000000,5255.000000,5255.000000,5255.000000,5255.000000
mean,2017.859372,1.938154,8.431066,0.108278,0.133587,0.301998,0.098763,0.105043,0.044148,0.010847,...,0.000761,0.392959,0.307136,0.001713,0.031018,0.006089,0.126356,0.008563,19.273454,16.527117
std,4.241768,0.743056,0.679967,0.310761,0.340240,0.459168,0.298372,0.306638,0.205444,0.103592,...,0.027582,0.488454,0.461350,0.041353,0.173383,0.077804,0.332282,0.092150,20.103813,17.158101
min,1987.000000,0.000000,3.440000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,2016.000000,1.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000
50%,2019.000000,2.000000,8.540000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,12.000000
75%,2021.000000,2.000000,8.970000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,44.000000,24.000000
max,2023.000000,3.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,120.000000,185.000000


In [27]:
data.to_csv('preprocessed_data.csv')